### This is a one time script to assign ONS output area populations to their centroid lat/long coordinates. 

It is used when estimating the population in each isochrone.

In [ ]:
# Import modules and set paths
import pandas as pd
from pathlib import Path
from pyproj import Transformer
ROOT = Path("../")
ROOT.resolve()

In [2]:
def convert_to_latlong(df, x_coord_name:str, y_coord_name:str, from_crs="EPSG:27700", to_crs="EPSG:4326"):
    '''
    Function to convert a single row of Easting/Northing to Lat/Long

    Parameters
    ----------
        df: pandas.DataFrame
        x_coord_name: str
        y_coord_name: str
    Returns
    -------
        df: pandas.DataFrame
            Dataframe with converted coordinates.
    '''
    # Example: OSGB36 (EPSG:27700) to WGS84 (EPSG:4326)
    transformer = Transformer.from_crs(from_crs, to_crs, always_xy=True)
    copy = df.copy()

    for i, row in copy.iterrows():
        longitude, latitude = transformer.transform(row[x_coord_name], row[y_coord_name])
        df.loc[i, 'latitude'] = round(latitude, 5)
        df.loc[i, 'longitude'] = round(longitude, 5)
    
    return df

In [4]:
# Read the centroids
centroids = pd.read_csv(ROOT / "data/population/output_area_centroids_2021.csv", usecols=['OA21CD', 'x', 'y'])
# Convert them to lat, longs
converted = convert_to_latlong(centroids, 'x', 'y')
# Read the populations
pop = pd.read_excel(ROOT / "data/population/sapeoatablefinal2022v2.xlsx", sheet_name='Mid-2022 OA 2021', header=3, usecols=['OA 2021 Code', 'Total'])
# Rename the column for merging
pop.rename(columns={'OA 2021 Code': 'OA21CD'}, inplace=True)
# Merge the centroid and population dataframes
merged = pop.merge(converted, on='OA21CD', how='inner')
# Slice only columns we need.
merged = merged.loc[:, ['OA21CD', 'latitude', 'longitude', 'Total']]
# Write file
merged.to_csv(ROOT / 'data/population/OA21CD_population.csv', index=False)